In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import warnings

In [2]:
# 1. Cargando de datos y preprocesamiento
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    data = pd.read_csv('Books.csv', low_memory=False)
data = data.sample(frac=0.1, random_state=1).reset_index(drop=True)  # Usando el 10% de los datos

In [3]:
# 2. Creación de perfiles de contenido
data['Content'] = data['Book-Title'] + ' ' + data['Book-Author'] + ' ' + data['Year-Of-Publication'].astype(str) + ' ' + data['Publisher']
data['Content'].fillna('ValorPredeterminado', inplace=True)

In [4]:
# 3. Vectorizando el texto (TF-IDF)
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(data['Content'])

In [5]:
# 4. Calculando la similitud del texto (similitud coseno)
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [6]:
# 5. Generación de recomendaciones
def get_recommendations(book_title, cosine_sim=cosine_sim):
    if book_title in data['Book-Title'].values:
        idx = data[data['Book-Title'] == book_title].index[0]  # Obtiene el índice del libro coincidente
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]  # Recomienda los 10 libros más similares
        book_indices = [i[0] for i in sim_scores]
        return data['Book-Title'].iloc[book_indices]
    else:
        return pd.Series([])  # Devuelve una Serie vacía si el libro no se encuentra en el conjunto de datos

In [7]:
# Prueba con un libro de ejemplo
libro = "Skinned Alive: Stories"
if libro in data['Book-Title'].values:
    recommendations = get_recommendations(libro)
    print("Recomendaciones para el libro '{}':".format(libro))
    print(recommendations)
else:
    print(f"'{libro}' no se encuentra en el conjunto de datos.")

Recomendaciones para el libro 'Skinned Alive: Stories':
4788                      Fanny: A Fiction (White, Edmund)
17816                                     Forgetting Elena
5008                                  Sketches from Memory
6696                    The Enchanted Land (Romance Alive)
26573      To the Wedding: A Novel (Vintage International)
19116                                          Another You
470                        Our Paris: Sketches from Memory
8533          River Dogs: Stories (Vintage Contemporaries)
18308    The Safety of Objects: Stories (Vintage Contem...
10586                      Nightrose (Romance Alive Audio)
Name: Book-Title, dtype: object
